Name : Sayfre Anderson, Howard Wen , Vamsi Sakhamuri

E-mail : ..., ..., vamsi@ischool.berkeley.edu

Class Name : W261-3

Week Number : 3

Date of submission : 02/2/2016

HW3.0.

What is a merge sort? Where is it used in Hadoop?

How is  a combiner function in the context of Hadoop? 

Give an example where it can be used and justify why it should be used in the context of this problem.

What is the Hadoop shuffle?

ANSWER 3.0

* Merge-sort is a sorting algorithm with runs in O(nlog(n)) time. In hadoop, it is used in between the map outputs and 
the reduce inputs. Output from each map output is sorted by the key and then merged with the other sorted outputs from other 
map outputs.

* Combiner function is used after the map output is generated. The values from the same keys are then aggregated to reduce network communication in their transfer to the reduce nodes. Network communication can be a bottleneck and a combiner helps alleviate this problem. 
Whether a combiner function is run or not is entirely dependent on the execution framework. To be totally sure that intermediate map outputs are combined, in-mapper combining function can be employed. This ensures that aggregation will happen 100% before being shuffled across the network to the various reduce nodes.

* An example where combiners (either the optional one provided by the execution framework or the in-mapper combiner) is helpful would be the word count application.
Without any sort of combiners, we would be transfer the same key multiple times within the same map task.

    For example, if the map input is "hello hello hi hi hello hello", 

    Without combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,1
    
    hello,1
    
    hi,1
    
    hi,1
    
    hello,1
    
    hello,1

    With combiners, the following <key,value> pairs will be emitted across the network and into the reduce nodes:

    hello,4
    
    hi,2

    This is an incredible reduction in network traffic.
    

* Hadoop Shuffle: Map-reduce makes the guarantee that the input to every reducer is sorted by the key. And that the <key,value> pairs with the same key are routed to the same reducer. The process by which the system performs this sort and merge (across various mappers) and transfers the map outputs to the reducers is known as the hadoop shuffle.

HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)

Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce 
stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the 
jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

In [3]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/start-dfs.sh

16/01/31 17:30:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /Users/Vamsi/Downloads/hadoop-2.7.1/logs/hadoop-Vamsi-namenode-Vamsis-MacBook-Air.local.out
localhost: datanode running as process 812. Stop it first.
Starting secondary namenodes [0.0.0.0]
0.0.0.0: secondarynamenode running as process 933. Stop it first.
16/01/31 17:31:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/mr-jobhistory-daemon.sh --config /Users/Vamsi/Downloads/hadoop-2.7.1/etc/hadoop/ start historyserver 

In [198]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -rm -r -f  /user/vamsi/hw3

16/02/01 14:17:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 14:17:48 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/vamsi/hw3


In [199]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -mkdir -p /user/vamsi/hw3

16/02/01 14:18:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [200]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put Consumer_Complaints.csv /user/vamsi/hw3

16/02/01 14:18:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [49]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
#Grabbing the header names
header = next(csv_iter)

for line in csv_iter:
    if(len(line)==len(header)):   #continue only if all the rows have all the entries
        if(line[1].lower()=="debt collection"):
            #print "%s\t1" %line[1]
            sys.stderr.write("reporter:counter:Product,Debt,1\n")
        elif(line[1].lower() == "mortgage"):
            #print "%s\t1" %line[1]   
            sys.stderr.write("reporter:counter:Product,Mortgage,1\n")
        else:
            #print "Other\t1" 
            sys.stderr.write("reporter:counter:Product,Other,1\n")

Overwriting mapper.py


In [ ]:
!chmod a+x mapper.py

In [26]:
!cat Consumer_Complaints.csv | python mapper.py >test_3_0

In [40]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

#NOTE: THIS REDUCER IS NOT REQUIRED FOR 3.1 . IT WAS WRITTEN JUST TO CHECK WHETHER THE COUNTERS WERE RETURNING THE 
#RIGHT VALUE. WE ARE USING AN IDENTITY REDUCER
count = 0

prev_string = None

for line in sys.stdin:
    line_s = re.split(r'[\t]',line)
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0

    count += 1
    prev_string = line_s[0]
print prev_string,count  

Overwriting reducer.py


In [ ]:
!chmod a+x reducer.py

In [7]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/vamsi/hw3/Consumer_Complaints.csv \
-output /user/vamsi/hw3/output_3_1 \
-mapper mapper.py \
-reducer org.apache.hadoop.mapred.lib.IdentityReducer 

16/01/31 17:33:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 17:33:24 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 17:33:24 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 17:33:24 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 17:33:24 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 17:33:25 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 17:33:25 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/31 17:33:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local171273095_0001
16/01/31 17:33:25 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/31 17:33:25 INFO mapreduce.Job: Running job: job

In [8]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_1/part-00000 

16/01/31 17:47:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [61]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/sbin/stop-dfs.sh

16/01/31 17:22:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: no datanode to stop
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
16/01/31 17:22:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [9]:
!echo "foo foo quux labs foo bar quux" >input_3_2

In [10]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put input_3_2 /user/vamsi/hw3

16/01/31 18:06:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [86]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

for line in sys.stdin:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    line_s = re.split(r'[\s]',line)
    for l in line_s:
        print "%s\t1" %l.strip()

Overwriting mapper.py


In [21]:
!chmod a+x mapper.py

In [89]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")

for line in sys.stdin:
    line_s = re.split(r'[\t]',line)
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0

    count += 1
    prev_string = line_s[0]
print prev_string,count  

Overwriting reducer.py


In [22]:
!chmod a+x reducer.py

In [90]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/vamsi/hw3/input_3_2 \
-output /user/vamsi/hw3/output_3_2_1Amm \
-mapper mapper.py \
-reducer reducer.py 

16/01/31 21:51:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 21:51:47 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 21:51:47 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 21:51:47 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 21:51:48 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 21:51:48 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 21:51:48 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/01/31 21:51:48 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1297211590_0001
16/01/31 21:51:48 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/31 21:51:48 INFO mapreduce.Job: The url to track the 

In [55]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_1/part-00000

16/01/31 18:45:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
quux 2	


In [57]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_1/part-00001

16/01/31 18:46:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 1	
foo 3	


In [58]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_1/part-00002

16/01/31 18:46:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bar 1	


In [59]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_1/part-00003

16/01/31 18:46:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
labs 1	


In [201]:
!tail -n +2 Consumer_Complaints.csv > Consumer_Complaints_no_header.csv

In [202]:
!split -l 78228 Consumer_Complaints_no_header.csv

In [203]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put xa* /user/vamsi/hw3

16/02/01 14:19:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [305]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3].strip())
        for w in issues:
            print "%s\t1" %w

Overwriting mapper.py


In [306]:
!chmod a+x mapper.py

Overwriting combiner.py


In [113]:
!chmod a+x combiner.py

In [309]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0

    count += 1
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [71]:
!chmod a+x mapper.py

In [310]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/vamsi/hw3/xa* \
-output /user/vamsi/hw3/output_3_2_2Bill \
-mapper mapper.py \
-reducer reducer.py 

16/02/01 16:26:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 16:26:20 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 16:26:20 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 16:26:20 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 16:26:21 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/01 16:26:21 INFO mapreduce.JobSubmitter: number of splits:4
16/02/01 16:26:21 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 16:26:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1154581649_0001
16/02/01 16:26:21 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 16:26:21 INFO mapred.LocalJobRunner: OutputC

In [311]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_2Bill/part-00000 | tail 

16/02/01 17:01:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
repay 1647	
service 1518	
the 6248	
to 8401	
transfer 597	
unable 3821	
verification 7655	
was 274	
wrong 71	
your 3844	


In [212]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_2BDC/part-00001

16/02/01 14:48:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 4	
ATM 2422	
Cancelling 2795	
Communication 8671	
Dealing 1944	
Improper 4966	
Loan 107254	
Payment 92	
Problems 9484	
Shopping 672	
Taking 4206	
Transaction 1098	
Underwriting 2774	
advance 240	
amount 98	
apply 118	
bank 202	
card 4405	
collect 17972	
costs 4350	
credited 92	
decision 2774	
didn't 925	
disclosures 64	
fee 3198	
funds 5663	
get 4357	
illegal 2964	
investigation 4858	
lease 6337	
lender 2165	
line 1732	
loan 12376	
marketing 1193	
missing 64	
modification 70487	
monitoring 1453	
mortgage 8625	
my 10731	
originator 8625	
out 1242	
plans 350	
rate 3431	
report 34903	
reporting 6559	
scam 566	
score 4357	
sending 3226	
servicing 36767	
statements 3621	
transaction 387	
withdrawals 10555	


In [213]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_2BDC/part-00002

16/02/01 14:48:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
APR 3431	
Arbitration 168	
Bankruptcy 222	
Billing 8158	
Convenience 75	
Credit 14768	
Deposits 10555	
Disclosure 7655	
False 3621	
Overlimit 127	
Received 118	
Repaying 3844	
Settlement 4350	
Unsolicited 640	
an 2964	
and 16448	
being 5663	
broker 8625	
changes 350	
charges 131	
credit 40483	
delay 243	
determination 1490	
did 139	
escrow 36767	
expect 807	
fees 807	
foreclosure 70487	
not 18477	
payments 39993	
process 5505	
promised 274	
protection 4139	
receive 139	
received 98	
relations 1367	
representation 3621	
servicer 1944	
statement 1220	
tactics 8671	
terms 350	
theft 3276	
threatening 2964	
use 1477	
you 3821	


In [214]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_2BDC/part-00003

16/02/01 14:48:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Advertising 1193	
Application 8868	
Balance 597	
Charged 878	
Collection 1907	
Customer 2734	
Embezzlement 3276	
Forbearance 350	
Fraud 3842	
Getting 291	
Identity 3276	
Late 1797	
Managing 5006	
Money 274	
Other 7886	
Privacy 240	
Rewards 1002	
Using 2422	
action 2964	
amt 71	
are 3821	
attempts 17972	
charged 98	
collection 70487	
contact 3710	
day 71	
debit 2422	
decrease 1149	
dispute 904	
identity 1453	
increase 1149	
info 3553	
information 29069	
interest 4238	
issues 538	
low 5663	
money 3365	
opening 16205	
pay 3821	
practices 1003	
receiving 3226	
sharing 3489	
stop 131	
when 4095	
with 1944	


In [314]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s\t1" %w

Overwriting mapper.py


In [ ]:
!chmod a+x mapper.py

In [316]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:COMBINER,combiner_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s\t%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s\t%s" %(prev_string,count)

Overwriting combiner.py


In [ ]:
!chmod a+x combiner.py

In [317]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[\t]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [318]:
!chmod a+x reducer.py

In [319]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=4 \
-input /user/vamsi/hw3/xa* \
-output /user/vamsi/hw3/output_3_2_3aaa \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/01 17:05:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:05:16 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:05:16 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:05:16 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:05:16 INFO mapred.FileInputFormat: Total input paths to process : 4
16/02/01 17:05:17 INFO mapreduce.JobSubmitter: number of splits:4
16/02/01 17:05:17 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 17:05:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local503180066_0001
16/02/01 17:05:17 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 17:05:17 INFO mapred.LocalJobRunner: OutputCommitte

In [325]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

csv_iter = csv.reader(sys.stdin)
    
for line in csv_iter:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    if(len(line)==14):   #continue only if all the rows have all the entries
        issues = re.split(r'[\s,./]+',line[3])
        for w in issues:
            print "%s,1" %w

Overwriting mapper.py


In [230]:
!chmod a+x mapper.py

In [323]:
%%writefile combiner.py
#!/usr/bin/python
import sys
import re

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:COMBINER,combiner_calls,1\n")
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print "%s,%s" %(prev_string,count)
        count = 0
    count += 1
    prev_string = line_s[0]
print "%s,%s" %(prev_string,count)

Overwriting combiner.py


In [ ]:
!chmod a+x combiner.py

In [324]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
import csv

count = 0

prev_string = None

for line in sys.stdin:
    sys.stderr.write("reporter:counter:REDUCER,reducer_calls,1\n")
    line_s = re.split(r'[,]',line.strip())
    
    if((prev_string!=None) and (prev_string != line_s[0])):
        print prev_string,count
        count = 0
    count += int(line_s[1])
    prev_string = line_s[0]
print prev_string,count

Overwriting reducer.py


In [ ]:
!chmod a+x reducer.py

In [328]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-input /user/vamsi/hw3/Consumer_Complaints.csv \
-output /user/vamsi/hw3/output_3_2_4HAIL \
-mapper mapper.py \
-combiner combiner.py \
-reducer reducer.py 

16/02/01 17:47:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:47:56 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:47:56 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:47:56 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:47:57 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:47:57 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:47:57 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 17:47:57 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local125708255_0001
16/02/01 17:47:57 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 17:47:57 INFO mapred.LocalJobRunner: OutputCo

In [330]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_4HAIL/part-00000 > reduce_3_2_4

16/02/01 17:51:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [339]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
import csv

for line in sys.stdin:
    sys.stderr.write("reporter:counter:MAPPER,mapper_calls,1\n")
    line = re.split(r'[\s]+',line)
    print "%s,%s" %(line[0],line[1])

Overwriting mapper.py


In [344]:
%%writefile reducer.py
#!/usr/bin/python
import sys

for line in sys.stdin:
    print line.strip()

Overwriting reducer.py


In [332]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -put reduce_3_2_4 /user/vamsi/hw3

16/02/01 18:00:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [345]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop jar /Users/Vamsi/Downloads/hadoop-2.7.1/bin/hadoop-*streaming*.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D stream.map.output.field.separator=, \
-D stream.num.map.output.key.fields=2 \
-D map.output.key.field.separator=, \
-D mapred.text.key.comparator.options=-k2,2nr \
-D mapred.reduce.tasks=1 \
-input /user/vamsi/hw3/reduce_3_2_4 \
-output /user/vamsi/hw3/output_3_2_4HAILONES \
-mapper mapper.py \
-reducer reducer.py

16/02/01 18:05:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 18:05:40 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 18:05:40 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 18:05:40 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 18:05:40 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 18:05:40 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 18:05:40 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
16/02/01 18:05:40 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/01 18:05:40 INFO Configuration.deprecation: mapr

In [346]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_4HAILONES/part-00000 | head -n 50

16/02/01 18:06:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Loan,107254	
collection,70487	
modification,70487	
foreclosure,70487	
account,40893	
or,40508	
credit,40483	
payments,39993	
escrow,36767	
servicing,36767	
report,34903	
Incorrect,29133	
information,29069	
on,29069	
debt,26531	
not,18477	
owed,17972	
collect,17972	
Cont'd,17972	
attempts,17972	
Account,16555	
and,16448	
closing,16205	
management,16205	
opening,16205	
Credit,14768	
of,13983	
loan,12376	
my,10731	
Deposits,10555	
withdrawals,10555	
Problems,9484	
Application,8868	
tactics,8671	
Communication,8671	
mortgage,8625	
originator,8625	
broker,8625	
to,8401	
Billing,8158	
Other,7886	
Disclosure,7655	
verification,7655	
disputes,6938	
reporting,6559	
lease,6337	
the,6248	
funds,5663	
low,5663	
being,5663	


In [348]:
!/Users/Vamsi/Downloads/hadoop-2.7.1/bin/hdfs dfs -cat /user/vamsi/hw3/output_3_2_4HAILONES/part-00000 | tail -n 10

16/02/01 18:06:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Payment,92	
Convenience,75	
checks,75	
day,71	
amt,71	
wrong,71	
disclosures,64	
missing,64	
,4	
Issue,1	


HW3.3. Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs